In [3]:
import pandas as pd
from pandas import ExcelWriter
import pandas as pd
import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")
def Del_string(xstr):
    xstrc = xstr.strip().strip("市").strip("省")
    if len(xstrc)>4:
        xstrc = xstrc[0:2]
    return xstrc

dic = { '申请单量': pd.Series.nunique,'授信单量': np.sum,\
       '用信单量': np.sum ,'授信余额': np.sum ,'授信总金额': np.sum ,'放款金额': np.sum}

dic_risk_m1 = {  '用信单量': np.sum ,'放款金额': np.sum,'M1_单量': np.sum,'M1_金额': np.sum}

dic_risk_mk = {  '用信单量': np.sum ,'放款金额': np.sum,'M1+_单量': np.sum,'M1+_金额': np.sum}



def calculate_col(df):
    try:
        df['授信率']= df['授信单量']/(df['申请单量'])
        df['通过率'] = df['用信单量'] / (df['申请单量'])
        df['用信率']= df['用信单量']/(df['授信单量'])   
        df['人均贷款本金']= df['放款金额']/(df['用信单量'])
        df['人均授信金额']= df['授信总金额']/(df['授信单量'])
  
    except ZeroDivisionError:
        pass
    return df


def row_rank(col):
    if  '全省' in col:
        return 1
    if  '大区汇总' in col:
        return 2
    if  '全国' in col:
        return 3
    else:
        return 0

In [4]:
yesterday = datetime.date.today()-datetime.timedelta(days=1)#返回时间格式
two_days_ago = datetime.date.today()-datetime.timedelta(days=2)
pass_day = yesterday.day
current_month = yesterday.strftime('%Y-%m')
total_day = 31


detail_file_name = '么么贷订单明细表/雪球字段要求.xlsx'
sheet_file_name = '明细'

#mer_file_name = '么么贷门店绑定人员明细表/么么贷门店绑定人员明细'+datetime.date.today().strftime('%m%d')+'.xlsx'
mer_file_name = '么么贷门店绑定人员明细表/么么贷门店绑定人员明细1203.xlsx'
mer_sheet_name = '门店绑定明细'

pos_file_name = '马上对应门店销售数据/马上美业商户销量.xlsx'
pos_sheet_name = 'Sheet'


path = 'C:/Users/qilong.zhang/Desktop/memedai/'


define_upload = pd.read_excel(path + detail_file_name,sheet_file_name,index='false',converters = {"biz_prov": Del_string, "biz_city": Del_string})

mimme_mer_detl = pd.read_excel(path + detail_file_name,'商户明细',index='false',converters = {"biz_prov": Del_string, "biz_city": Del_string})
#mimme_mer_detl = pd.read_excel(path + '么么贷订单明细表/雪球商户明细-20190826.xlsx',index='false',converters = {"biz_prov": Del_string, "biz_city": Del_string})


mer_upload = pd.read_excel(path+mer_file_name,mer_sheet_name,index='false',converters = {"城市": Del_string})

msxf_mer_loan = pd.read_excel(path+pos_file_name,pos_sheet_name,index='false',converters = {"省份": Del_string, "城市": Del_string})


sa_info =  pd.read_excel(path+'马上对应门店销售数据/SA马上销量.xlsx',index='false',converters = {"省份": Del_string, "城市": Del_string})

area_pro_city =  pd.read_excel(path+'大区-省份-城市.xlsx',index='false',converters = {"省份": Del_string, "城市": Del_string})


#休假人员大于10天
sa_vacation =  pd.read_excel(path+ '2019年8月休假名单.xlsx',index='false')


define_upload['appl_create_dt'] = define_upload['appl_create_dt'].astype(str).str[0:10]
define_upload['ord_date'] = define_upload['ord_create_dt'].astype(str).str[0:10]

define_upload['appl_create_month'] = define_upload['appl_create_dt'].astype(str).str[0:7]
define_upload['授信总金额'] = define_upload[['loan_amt','credit_bal_amt_on']].sum(axis = 1)
define_upload.loc[define_upload['biz_city'] == '辖区','biz_city'] = define_upload['biz_prov']
define_upload.loc[define_upload['mer_no'] == 7941,'mer_no'] = 6935
define_upload['M1+_单量'] = (define_upload['cpd'] >30).map({True:1,False :0})
define_upload['M1+_金额'] = define_upload['M1+_单量']*define_upload['total_bal']

define_upload['M1_单量'] = (define_upload['dpd_seg_lv'] == 'M1').map({True:1,False :0})
define_upload['M1_金额'] = define_upload['M1_单量']*define_upload['total_bal']



define_upload.rename(columns={ 'biz_prov':'省份','biz_city':'城市',
                              'loan_amt': '放款金额','apro_status': '用信单量',
                              'appl_no': '申请单量','apro_result': '授信单量',
                              'credit_bal_amt_on': '授信余额','mer_no': '么么商户代码'}, inplace=True)

print('文件读取完成！',datetime.datetime.now())

文件读取完成！ 2019-12-04 09:13:59.437528


In [5]:
#商户清单

#可做单商户清单
mimme_mer_act  = mimme_mer_detl[mimme_mer_detl['fx_status'] == 1]['mer_no'].drop_duplicates()

mer_sa_fix = pd.merge(area_pro_city[area_pro_city['是否保留城市']=='是'],mer_upload[mer_upload.销售ID.notnull()],how = 'inner',on = '城市')

mer_sa_fix = pd.merge(mer_sa_fix,mimme_mer_act,how = 'inner',left_on = '门店编码',right_on = 'mer_no',suffixes = ('','_y'))


meme_mer_loan_m = define_upload[define_upload.appl_create_month == current_month].groupby(\
                                                             '么么商户代码',as_index= False).agg({ \
                                                            '申请单量': pd.Series.nunique,'授信单量': np.sum,\
                                                           '用信单量': np.sum ,'授信余额': np.sum ,'授信总金额': np.sum, \
                                                           '放款金额': np.sum})
#出单商户清单
mer_pay_detl= define_upload[(define_upload.appl_create_month == current_month)& (define_upload['用信单量']==1 )].groupby(\
                                                             '么么商户代码',as_index= False).agg({ 'ord_date': pd.Series.nunique})

meme_mer_loan_m = pd.merge(meme_mer_loan_m,mer_pay_detl,how = 'left',on = '么么商户代码')


meme_mer_loan_m.rename(columns = {'ord_date':'米么出单天数'},inplace =True)

meme_mer_loan_d = define_upload[define_upload.appl_create_dt == str(yesterday)].groupby('么么商户代码',as_index= False).agg(dic)
                                                                                                    
meme_mer_loan_m = calculate_col(meme_mer_loan_m)
meme_mer_loan_d = calculate_col(meme_mer_loan_d)
                                                                                                     
mer_all_data1 = pd.merge(mer_sa_fix[['大区','省份','城市','门店编码','门店名称（么么贷）','雪球商户代码','雪球商户名称']],\
                         meme_mer_loan_m, how='left', left_on='门店编码',right_on = '么么商户代码',suffixes = ('','_y'))
mer_all_data2 = pd.merge(mer_all_data1, meme_mer_loan_d, how='left', left_on='门店编码',right_on = '么么商户代码',\
                         suffixes = ('(么么)_m','(么么)_d'))

mer_all_data = pd.merge(mer_all_data2,msxf_mer_loan, how='left', left_on='雪球商户代码',right_on = '雪球商户代码',suffixes = ('','_y'))

listBins1 = [0,1,200000, 500000, 1000000, 2000000,float('inf')]
Bins_lable2 = ['0', '(0,20万]', '(20,50万]','(50,100万]', '(100,200万]','200万以上']
mer_all_data['本月米么商户产能等级'] = pd.cut((mer_all_data['放款金额(么么)_m']/pass_day*total_day).fillna(0),right=True,bins= listBins1,labels= Bins_lable2 ,include_lowest=False)

#print(mer_all_data2)
#mer_all_data['是否么么贷合作商户'] = mer_all_data['门店编码'].isnull().map({True:'否',False:'是'})
#moth_data_all['是否么么贷合作商户'] = np.where(moth_data_all['门店编码'].isnull(),'否','是')
try:
    mer_all_data['么么贷放款占比_m'] = mer_all_data['放款金额(么么)_m']/ (mer_all_data['放款金额(马上)_m']+mer_all_data['放款金额(么么)_m'])
    mer_all_data['么么贷放款占比_d'] = mer_all_data['放款金额(么么)_d']/ (mer_all_data['放款金额(马上)_d']+mer_all_data['放款金额(么么)_d'])
except ZeroDivisionError:
    pass

col_name = ['大区', '省份', '城市','门店编码','门店名称（么么贷）','雪球商户代码', '雪球商户名称','18年四季度月均销量(马上)','近三月月均销量(马上)','本月米么商户产能等级','米么出单天数',\
            '申请单量(么么)_m','授信单量(么么)_m', '用信单量(么么)_m', '授信率(么么)_m', '用信率(么么)_m','通过率(么么)_m',\
           '授信总金额(么么)_m', '放款金额(么么)_m', '么么贷放款占比_m','申请单量(么么)_d', '授信单量(么么)_d', '用信单量(么么)_d',\
            '授信率(么么)_d',  '用信率(么么)_d','通过率(么么)_d','授信总金额(么么)_d','放款金额(么么)_d','么么贷放款占比_d',\
            '申请单量(马上)_m','通过单量(马上)_m', '通过率(马上)_m', '通过金额(马上)_m',  '放款单量(马上)_m', '放款金额(马上)_m', \
            '申请单量(马上)_d','通过单量(马上)_d', '通过率(马上)_d', '通过金额(马上)_d', '放款单量(马上)_d', '放款金额(马上)_d']

mer_all_data = mer_all_data.reindex(columns = col_name ).sort_values(by = '放款金额(么么)_m',ascending = False).fillna({'18年四季度月均销量(马上)':'0',\
                                                                                                                 '近三月月均销量(马上)':'0','本月米么商户产能等级':'0'})
#mer_all_data.fillna('-',inplace = True)

print('商户明细文件处理完成！',datetime.datetime.now())




#SA_id字段处理
columns_name = mer_sa_fix.columns.tolist()
sa_id_col = [item for i,item in enumerate(columns_name) if item.find('销售')!=-1]
mer_sa_fix[sa_id_col] = mer_sa_fix[sa_id_col].astype(float)
sa_id_data = mer_sa_fix[['城市']+ sa_id_col]
sa_id_melt = pd.melt(sa_id_data,id_vars = ['城市'])
mm_sa_id = sa_id_melt.iloc[:,2].drop_duplicates()
#在职人员明细表
sa_info_all = pd.merge(sa_info,mm_sa_id,how = 'left',left_on = 'salman_id' ,right_on = 'value')

#米么出单SA
mime_sa_sales  =  define_upload[(define_upload.appl_create_month == current_month) & \
                                           (define_upload.用信单量 == 1)].groupby('sa_id',as_index = False)[['放款金额','用信单量']].sum()

all_sa_sales = pd.merge(sa_info_all,mime_sa_sales,how = 'left',left_on = 'salman_id' ,right_on = 'sa_id')

all_sa_sales['在职SA是否绑定米么'] = (all_sa_sales['salman_id'] == all_sa_sales['value']).map({True :1,False : 0})
all_sa_sales['米么SA是否出单'] = (all_sa_sales['salman_id'] == all_sa_sales['sa_id']).map({True :1,False : 0})
all_sa_sales['两边不出单'] = (((all_sa_sales['本月马上销量']== 0) | (all_sa_sales['本月马上销量'].isnull())) &\
                         (all_sa_sales['sa_id'].isnull())).map({True :1,False : 0})

all_sa_sales['是否在马上出单'] = ((all_sa_sales['本月马上销量'].notnull())| (all_sa_sales['本月马上销量']>0)).map({True :1,False : 0})
all_sa_sales['本月预计销量'] = all_sa_sales['放款金额']/pass_day* total_day

all_sa_sales =pd.merge(all_sa_sales,sa_vacation,how = 'left',left_on = '工号' ,right_on = '工号')
all_sa_sales['是否休假(产假或其他)'] = (all_sa_sales['休假天数(产假或其他)'].notnull()).map({True :'是',False : '否'})




listBins = [0,1,90000, 180000, 270000, 360000,float('inf')]
Bins_lable = ['零产', '低产', '中低产','中产', '中高产','高产']
all_sa_sales['米么SA产能等级'] = pd.cut(all_sa_sales['本月预计销量'].fillna(0),right=False,bins= listBins,labels= Bins_lable ,include_lowest=True)
all_sa_sales['18年四季度SA产能等级(马上)'] = pd.cut(all_sa_sales['18年四季度月均销量(马上)'].fillna(0),right=False,bins= listBins,labels= Bins_lable ,include_lowest=True)
all_sa_sales['近三月SA产能等级(马上)'] = pd.cut(all_sa_sales['近三月月均销量(马上)'].fillna(0),right=False,bins= listBins,labels= Bins_lable ,include_lowest=True)


all_sa_sales = all_sa_sales.reindex(columns=['区域', '省份', '城市','是否保留城市', '工号', 'salman_id','salman_status','是否休假(产假或其他)',\
                                             '在职SA是否绑定米么','两边不出单','是否在马上出单','米么SA是否出单','放款金额',\
                                             '用信单量', '本月预计销量','米么SA产能等级','本月马上单量','本月马上销量','18年四季度月均销量(马上)',\
                                             '近三月月均销量(马上)','18年四季度SA产能等级(马上)','近三月SA产能等级(马上)'])

all_sa_sales.rename(columns = {'salman_status':'SA状态','放款金额':'米么销量','用信单量':'米么单量'},inplace = True)
#all_sa_sales[['马上单量','米么销量','米么单量','马上销量']] = all_sa_sales[['马上单量','米么销量','米么单量','马上销量']].fillna(0)

sa_detail = all_sa_sales[all_sa_sales['是否保留城市'] == '是']
sa_detail.to_excel(path + 'SA明细数据.xlsx',index  = False)
print('SA明细数据done')



city_sa_num = all_sa_sales[(all_sa_sales['是否保留城市'] == '是')&(all_sa_sales['是否休假(产假或其他)'] == '否')].groupby(
    ['区域','省份','城市','是否保留城市'],as_index = False) .agg(\
                                                     {'salman_id':pd.Series.nunique, '在职SA是否绑定米么':np.sum,\
                                                      '本月马上销量':np.sum,'两边不出单':np.sum,'是否在马上出单':np.sum,\
                                                      '米么SA是否出单':np.sum})
                                                                                                                                                                     

city_sa_num.rename(columns={'salman_id':'在职SA总人数','在职SA是否绑定米么':'米么绑定SA人数',
                            '两边不出单':'两边不出单SA人数','是否在马上出单':'马上出单SA人数'},inplace = True)
#city_sa_num--各城市激活SA办单情况

#各城市可做单商户数
city_mer_act_num = mer_sa_fix.groupby(['城市'],as_index=False).agg({'门店编码':pd.Series.nunique})

#各城市激活、出单商户数以及绑定SA人数
mer_sa_relationship = pd.merge(city_sa_num, city_mer_act_num, how='left', on='城市')
mer_sa_relationship.rename(columns = {'门店编码':'米么激活商户数'}, inplace=True)

#mer_sa_relationship =  pd.merge(all_mer_sa_num,all_mer_num,how='left', left_on=['区域','省份','城市'], right_on=['大区','省份','城市'])

print('商户绑定关系处理完成!',datetime.datetime.now())

#商户、人员信息分省、区、全国整合
prov_data = mer_sa_relationship.groupby(['区域','省份'],as_index= False).sum()
col_name2 =prov_data.columns.tolist()
col_name2 = col_name2[0:2]+ ['城市'] + col_name2[2:]
prov_data = prov_data.reindex(columns = col_name2,fill_value = '全省')
print('省份成功',datetime.datetime.now())

region_data = mer_sa_relationship.groupby(['区域'],as_index= False).sum()
col_name1 =region_data.columns.tolist()
col_name1 = col_name2[0:1]+ ['省份','城市'] + col_name2[1:]
region_data = region_data.reindex(columns = col_name2,fill_value = '大区汇总')
print('大区成功',datetime.datetime.now())

nation_data = mer_sa_relationship.drop(['区域','省份','城市','是否保留城市'],axis = 1).sum().to_frame().T
col_name = nation_data.columns.tolist()
col_name =  ['区域','省份','城市'] + col_name[0:]
nation_data = nation_data.reindex(columns = col_name,fill_value = '全国')
print('全国成功',datetime.datetime.now())
concat_data = pd.concat([mer_sa_relationship,prov_data,region_data,nation_data],axis=0)
print('合并成功',datetime.datetime.now())
#concat_data

#近30天米么每日销量
meme_sale_daliy = define_upload[define_upload['appl_create_dt']>=str(yesterday - datetime.timedelta(days=30))].groupby('appl_create_dt',as_index= False).agg(dic)
meme_sale_daliy = calculate_col(meme_sale_daliy)
#meme_sale_daliy = meme_sale_daliy.drop(['M1_单量','M1+_单量','M1_金额','M1+_金额'],axis = 1)
print('每日销量搞定',datetime.datetime.now())



#米么本月累计销量

#城市、省份、大区、全国统计
meme_city = define_upload[define_upload['appl_create_month'] == current_month].groupby(['城市'],as_index= False).agg(dic)
meme_city_t_1 = define_upload[define_upload['appl_create_dt'] == str(yesterday)].groupby(['城市'],as_index= False).agg({'放款金额': np.sum})
meme_city_t_2 = define_upload[define_upload['appl_create_dt'] == str(two_days_ago)].groupby(['城市'],as_index= False).agg({'放款金额': np.sum})
meme_city = pd.merge(meme_city,meme_city_t_1, how='left', left_on='城市',right_on = '城市',suffixes = ('','(昨天)'))
meme_city = pd.merge(meme_city,meme_city_t_2, how='left', left_on='城市',right_on = '城市',suffixes = ('','(前天)'))

meme_city_m = pd.merge(area_pro_city[['大区','省份','城市']],meme_city, how='left', on='城市',suffixes=('','_y'))
#print(meme_city_m.columns)

#出单sa/商户数
#city_pay_mer_num = define_upload[(define_upload.appl_create_month == current_month) & (define_upload.用信单量 == 1)].groupby(['城市'],as_index = False).agg({'么么商户代码':pd.Series.nunique,'sa_id':pd.Series.nunique})

#出单sa数,不含离职
city_sa_pay_num = sa_detail[sa_detail['米么SA是否出单']== 1].groupby(['城市'],as_index = False).agg({'salman_id':pd.Series.nunique})
#出单商户数，不含关闭商户
city_mer_pay_num = mer_all_data[~((mer_all_data['用信单量(么么)_m']==0)|mer_all_data['用信单量(么么)_m'].isnull())].groupby(['城市'],as_index = False).agg({'门店编码':pd.Series.nunique})


meme_city_m_a = pd.merge(meme_city_m,city_sa_pay_num,how='left', on='城市')
meme_city_m = pd.merge(meme_city_m_a,city_mer_pay_num,how='left', on='城市')

#print(meme_city_m.columns)
meme_city_m.rename(columns = {'门店编码':'米么出单商户数','salman_id':'米么出单SA人数'}, inplace=True)

mxsf_city_m = msxf_mer_loan.groupby(['城市'],as_index= False)['放款金额(马上)_m'].sum()
meme_city_m = pd.merge(meme_city_m,mxsf_city_m, how='left', on='城市')

商户明细文件处理完成！ 2019-12-04 09:14:00.098238
SA明细数据done
商户绑定关系处理完成! 2019-12-04 09:14:00.538064
省份成功 2019-12-04 09:14:00.548038
大区成功 2019-12-04 09:14:00.555020
全国成功 2019-12-04 09:14:00.559010
合并成功 2019-12-04 09:14:00.568980
每日销量搞定 2019-12-04 09:14:00.587929


In [6]:
#风控计算 

lastmonth_date = yesterday - relativedelta(months=1)#上月对应日期
two_month_ago_date = datetime.date.today() - relativedelta(months=2)#上上月对应日期

#m1每月风控分子、分母
risk_date = define_upload[define_upload['appl_create_dt']<= str(lastmonth_date)]
risk_m1 = risk_date.groupby(['城市','appl_create_month'],as_index = False).agg(dic_risk_m1)

#m1+ 每月风控分子、分母
risk_mk = risk_date[define_upload['appl_create_dt']<= str(two_month_ago_date)].groupby(['城市','appl_create_month'],as_index = False).agg(dic_risk_mk)

#合并m1/m1+数据
risk_m1_mk = pd.merge(risk_m1,risk_mk, how='left', on=['城市','appl_create_month'],suffixes=('_m1','_mk'))

#m1,m2累计风控分子、分母
risk_total = risk_m1_mk.groupby(['城市'],as_index = False).sum()

def col_rename(df):
    for i in df.columns:
        df.rename(columns = {i:i+'_累计'},inplace = True)
    return df
col_rename(risk_total) 

#将日期放到列上
risk_m1_mk_pivot = risk_m1_mk.pivot(index = '城市',columns = 'appl_create_month').reset_index()

city_sale_risk = pd.merge(meme_city_m,risk_total, how='left',left_on = '城市',right_on = '城市_累计')

city_sale_risk = pd.merge(city_sale_risk,risk_m1_mk_pivot,how = 'left',on = '城市')



print('城市月累计成功',datetime.datetime.now())


#省份
meme_prov_m = city_sale_risk.groupby(['大区','省份'],as_index= False).sum()

col_name =meme_prov_m.columns.tolist()
col_name = col_name[0:2]+ ['城市'] + col_name[2:]
meme_prov_m = meme_prov_m.reindex(columns = col_name,fill_value = '全省')
print('省份月累计搞定',datetime.datetime.now())

#大区
meme_region_m = city_sale_risk.groupby(['大区'],as_index= False).sum()
col_name =meme_region_m.columns.tolist()
col_name = col_name[0:1]+ ['省份','城市'] + col_name[1:]
meme_region_m = meme_region_m.reindex(columns = col_name,fill_value = '大区汇总')
print('大区月累计搞定',datetime.datetime.now())

#全国
meme_nation_m = city_sale_risk.drop(['大区','省份','城市'],axis = 1).sum().to_frame().T
col_name = meme_nation_m.columns.tolist()
col_name =  ['大区','省份','城市'] + col_name[0:]
meme_nation_m = meme_nation_m.reindex(columns = col_name,fill_value = '全国')
print('全国月累计搞定',datetime.datetime.now())

#米么城市、省份、大区、全国月累计数据合并
moth_all_data = pd.concat([city_sale_risk,meme_prov_m,meme_region_m,meme_nation_m],axis=0)
moth_all_data = calculate_col(moth_all_data)

#城市日报数据全集
all_data =  pd.merge(concat_data,moth_all_data,how='left', left_on=['区域','省份','城市'], right_on=['大区','省份','城市'])
print('数据集合并妥了',datetime.datetime.now())
#moth_all_data

城市月累计成功 2019-12-04 09:14:00.824330
省份月累计搞定 2019-12-04 09:14:00.834274
大区月累计搞定 2019-12-04 09:14:00.839261
全国月累计搞定 2019-12-04 09:14:00.880152
数据集合并妥了 2019-12-04 09:14:00.906113


In [7]:
#衍生字段
try:
    all_data['两边均未出单SA人数占比'] = all_data['两边不出单SA人数']/ all_data['在职SA总人数']
    all_data['么么贷放款占比'] = all_data['放款金额']/ (all_data['放款金额(马上)_m']+all_data['放款金额'])
    #all_data['激活商户数占比'] = all_data['米么激活商户数']/ all_data['可开发商户数量']
    all_data['出单商户数占比'] = all_data['米么出单商户数']/ (all_data['米么激活商户数'])
    all_data['商户产能'] = all_data['放款金额']/pass_day*total_day/ all_data['米么激活商户数']
    all_data['SA产能'] = all_data['放款金额']/pass_day* total_day/ all_data['米么绑定SA人数']

    all_data['米么出单SA人数占比'] = all_data['米么出单SA人数']/ all_data['米么绑定SA人数']

    all_data['昨日较前天增长率'] = all_data['放款金额(昨天)']/ all_data['放款金额(前天)'] - 1

    all_data['昨日较前天增长量'] =all_data['放款金额(昨天)'] - all_data['放款金额(前天)']
    all_data['昨日较前天拉动增长率'] = all_data['昨日较前天增长量'] / (float(all_data.loc[all_data['城市'] =='全国','放款金额(前天)']))
    all_data['昨日较前天增长贡献率'] =all_data['昨日较前天增长量']/ (abs(float(all_data.loc[all_data['城市'] =='全国','昨日较前天增长量'])))
    
    #累计风控数据
    all_data['M1_单量占比_累计'] = all_data['M1_单量_累计']/ all_data['用信单量_m1_累计']
    all_data['M1单量/总单量_累计'] = all_data['M1_单量_累计'].fillna(0).map(int).map(str)+ '|'+all_data['用信单量_m1_累计'].fillna(0).map(int).map(str)
    all_data['M1_金额占比_累计'] = all_data['M1_金额_累计']/ all_data['放款金额_m1_累计']
    all_data['M1金额/总金额_累计'] = all_data['M1_金额_累计'].fillna(0).map(int).map(str)+ '|'+all_data['放款金额_m1_累计'].fillna(0).map(int).map(str)    
    
    all_data['M1+_单量占比_累计'] = all_data['M1+_单量_累计']/ all_data['用信单量_mk_累计']
    all_data['M1+单量/总单量_累计'] = all_data['M1+_单量_累计'].fillna(0).map(int).map(str)+ '|'+all_data['用信单量_mk_累计'].fillna(0).map(int).map(str)
    all_data['M1+_金额占比_累计'] = all_data['M1+_金额_累计']/ all_data['放款金额_mk_累计']
    all_data['M1+金额/总金额_累计'] = all_data['M1+_金额_累计'].fillna(0).map(int).map(str)+ '|'+all_data['放款金额_mk_累计'].fillna(0).map(int).map(str)
 
    mon = ['7月','8月','9月','10月','11月']
    year_mon = ['2019-07','2019-08','2019-09','2019-10','2019-11']
    for i,j in zip(mon,year_mon):
 #i风控数据
        all_data['M1单量/总单量_'+i] = all_data[('M1_单量', j)].fillna(0).map(int).map(str)+ '|'+all_data[('用信单量_m1', j)].fillna(0).map(int).map(str)		
        all_data['M1_单量占比_'+i] = all_data[('M1_单量', j)]/ all_data[('用信单量_m1', j)]
        
        all_data['M1金额/总金额_'+i] = all_data[('M1_金额', j)].fillna(0).map(int).map(str)+ '|'+all_data[('放款金额_m1', j)].fillna(0).map(int).map(str)		
        all_data['M1_金额占比_'+i] = all_data[('M1_金额', j)]/ all_data[('放款金额_m1', j)]
    

        all_data['M1+单量/总单量_'+i] = all_data[('M1+_单量', j)].fillna(0).map(int).map(str)+ '|'+all_data[('用信单量_mk', j)].fillna(0).map(int).map(str)
        all_data['M1+_金额占比_'+i] = all_data[('M1+_金额', j)]/ all_data[('放款金额_mk', j)]

        all_data['M1+金额/总金额_'+i] = all_data[('M1+_金额', j)].fillna(0).map(int).map(str)+ '|'+all_data[('放款金额_mk', j)].fillna(0).map(int).map(str)
        all_data['M1+_单量占比_'+i] = all_data[('M1+_单量', j)]/ all_data[('用信单量_mk', j)]
        
except ZeroDivisionError:
    pass



#all_data.to_excel(path + 'kkkk.xlsx')
cols = ['累计','7月','8月','9月','10月','11月']
cols_risk = []
for i in cols:
    cols_risk += ['M1单量/总单量_'+i ,'M1_单量占比_'+i,'M1金额/总金额_'+i ,\
           'M1_金额占比_'+i,'M1+单量/总单量_'+i,'M1+_单量占比_'+i ,\
           'M1+金额/总金额_'+i ,'M1+_金额占比_'+i]

#数据字段排序整理
col_name = ['区域','省份','城市', '是否保留城市','米么激活商户数','米么出单商户数',
            '出单商户数占比', '在职SA总人数','米么绑定SA人数', '两边均未出单SA人数占比','米么出单SA人数占比',
            '申请单量', '授信单量','用信单量', '授信率', '用信率','通过率','授信总金额', '人均授信金额', '放款金额','么么贷放款占比', 
            '人均贷款本金','商户产能', 'SA产能','放款金额(昨天)','昨日较前天增长率','昨日较前天增长贡献率','昨日较前天拉动增长率'] + cols_risk

           
all_data = all_data.reindex(columns = col_name).fillna("-")

#大区-全国数据
moth_regoin_data =  all_data.loc[all_data['城市'].isin(['大区汇总', '全国'])]

#城市、省份排序
all_data['rank1'] = all_data.apply(lambda x: row_rank(x['城市']),axis = 1)

#排序后城市、省份、全国数据
moth_city_prov_data = all_data[all_data['城市']!='大区汇总'].sort_values(by = ['区域','省份','rank1'],ascending = [True,True,True]).drop(['rank1'],axis=1)

print('城市日报数据处理完成！',datetime.datetime.now())

城市日报数据处理完成！ 2019-12-04 09:14:01.139462


In [8]:
#数据导出
writer = pd.ExcelWriter( path + '么么贷-城市日报.xlsx')
moth_city_prov_data.to_excel(writer ,sheet_name='省份-城市',index  = False)
moth_regoin_data.to_excel(writer ,sheet_name='大区',index  = False)
mer_all_data[~((mer_all_data['用信单量(么么)_m']==0)|(mer_all_data['用信单量(么么)_m'].isnull()))].to_excel(writer , sheet_name = '出单商户明细',index  = False)
mer_all_data[(mer_all_data['用信单量(么么)_m']==0)|(mer_all_data['用信单量(么么)_m'].isnull())].to_excel(writer , sheet_name = '零单商户明细',index  = False)

sa_detail2 = sa_detail.copy(deep = True)
sa_detail2[['在职SA是否绑定米么','两边不出单','是否在马上出单','米么SA是否出单']] = \
sa_detail2[['在职SA是否绑定米么','两边不出单','是否在马上出单','米么SA是否出单']].astype(str).replace('1','是').replace('0','否')

sa_detail2[sa_detail2['米么SA是否出单']=='是'].to_excel(writer , sheet_name = '出单SA明细',index  = False)
sa_detail2[~(sa_detail2['米么SA是否出单']=='是')].to_excel(writer , sheet_name = '零单SA明细',index  = False)


meme_sale_daliy.to_excel(path + '么么贷-每日数据.xlsx',index  = False)
writer.save()
writer.close()
print('导出成功',datetime.datetime.now())

导出成功 2019-12-04 09:14:01.655084


In [9]:
#每日绑定SA人数追加
import openpyxl
region_sa_num = region_data[['区域','在职SA总人数','米么绑定SA人数','两边不出单SA人数','米么SA是否出单']]
region_sa_num.insert(0,'日期',str(yesterday))
list_data = np.array(region_sa_num).tolist()
wb = openpyxl.load_workbook(path+'各区出单SA人数.xlsx')
ws = wb['Sheet1']
for i in list_data:
    ws.append(i)
wb.save(path+'各区出单SA人数.xlsx')
print('done!',datetime.datetime.now())

done! 2019-12-04 09:14:04.028754


In [10]:
# 华南数据

#华南数据导出#发送加红
south_data = define_upload[define_upload['省份'].isin(['广东','福建','海南','江西'])]
south_daliy = south_data[south_data['appl_create_dt']>=str(yesterday - datetime.timedelta(days=30))].groupby('appl_create_dt',as_index= False).agg(dic)
south_daliy = calculate_col(south_daliy)


suoth_regoin_data = all_data[all_data['区域'] == '华南']
suoth_mer_data = mer_all_data[mer_all_data['大区'] == '华南']
 
    
writer = pd.ExcelWriter( path + '华南独立数据发送/么么贷-华南城市日报.xlsx')
suoth_regoin_data.to_excel(writer ,sheet_name='省份_城市',index  = False)

suoth_mer_data[~((suoth_mer_data['用信单量(么么)_m']==0)|(suoth_mer_data['用信单量(么么)_m'].isnull()))].to_excel(writer , sheet_name = '出单商户明细',index  = False)
suoth_mer_data[(suoth_mer_data['用信单量(么么)_m']==0)|(suoth_mer_data['用信单量(么么)_m'].isnull())].to_excel(writer , sheet_name = '零单商户明细',index  = False)

sa_detail2 = sa_detail[sa_detail['区域'] == '华南']
sa_detail2[['在职SA是否绑定米么','两边不出单','是否在马上出单','米么SA是否出单']] = \
sa_detail2[['在职SA是否绑定米么','两边不出单','是否在马上出单','米么SA是否出单']].astype(str).replace('1','是').replace('0','否')

sa_detail2[sa_detail2['米么SA是否出单']=='是'].to_excel(writer , sheet_name = '出单SA明细',index  = False)
sa_detail2[~(sa_detail2['米么SA是否出单']=='是')].to_excel(writer , sheet_name = '零单SA明细',index  = False)
south_daliy.to_excel(writer,sheet_name = '每日数据',index  = False)

writer.save()
writer.close()
print('华南数据导出成功',datetime.datetime.now())

'''#SMTP邮件
# -*- coding: UTF-8 -*-
import smtplib
import email.mime.multipart
import email.mime.text
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

def send_email(smtpHost, sendAddr, password, recipientAddrs, subject='', content=''):
    msg = email.mime.multipart.MIMEMultipart()
    msg['from'] = sendAddr
    msg['to'] = recipientAddrs
    msg['subject'] = subject
    content = content
    txt = email.mime.text.MIMEText(content, 'plain', 'utf-8')
    msg.attach(txt)

    # 添加附件
    part = MIMEApplication(open('C:\\Users\\qilong.zhang\\Desktop\\memedai\\华南独立数据发送\\么么贷-华南城市日报.xlsx','rb').read())
    part.add_header('Content-Disposition', 'attachment', filename="么么贷-华南城市日报.xlsx")
    msg.attach(part)

    smtp = smtplib.SMTP()
    smtp.connect(smtpHost, '25')
    smtp.login(sendAddr, password)
    smtp.sendmail(sendAddr, recipientAddrs, str(msg))
    print("邮件发送成功!",datetime.datetime.now())
    smtp.quit()

try:
    subject = '么么贷-华南城市日报'
    content = 'Dear加红:\n附件是么么贷-华南城市日报,数据截止'+ str(yesterday)
    send_email('smtp.qiye.aliyun.com', 'reportsend@qhsnowball.com', 'xxxx', 'jiahong.shi@zkj.com', subject, content)
except Exception as err:
    print(err)
'''

####exchange发邮件
from exchangelib import DELEGATE, Account, Credentials, Configuration, NTLM, Message, Mailbox, HTMLBody,FileAttachment
from exchangelib.protocol import BaseProtocol, NoVerifyHTTPAdapter
BaseProtocol.HTTP_ADAPTER_CLS = NoVerifyHTTPAdapter
def Send_Email(to,subject,body):
    creds = Credentials(
        username='datanreport',
        password='xxxxx'
    )
    config = Configuration(server='mail.zkjin.com', credentials=creds, auth_type=NTLM)
    account = Account(primary_smtp_address='datanreport@zkj.com', config=config, autodiscover=False, access_type=DELEGATE)
    msg = Message(
        account=account,
        subject=subject,
        body=HTMLBody(body),
        to_recipients = to,#收件人列表
        cc_recipients = []#抄送人列表
    )
    with open('C:\\Users\\qilong.zhang\\Desktop\\memedai\\华南独立数据发送\\么么贷-华南城市日报.xlsx','rb') as f:
        myfile = FileAttachment(name='么么贷-华南城市日报.xlsx', content= f.read())
    msg.attach(myfile)
    msg.send_and_save()

try:
    subject = '么么贷-华南城市日报'
    content = 'Dear加红:\n附件是么么贷-华南城市日报,数据截止'+ str(yesterday)
    Send_Email(['jiahong.shi@zkj.com'], subject, content)
    print("邮件发送成功!",datetime.datetime.now())
except Exception as err:
    print(err)    

华南数据导出成功 2019-12-04 09:14:04.324964
邮件发送成功! 2019-12-04 09:14:08.447975


128